In [1]:
import pandas as pd
import numpy as np

# 1

In [5]:
#1 preprocessing
data=pd.read_csv('penguins.csv')
row=[]
c=data.loc[:, data.isnull().any()].columns
for i in c:
    for j in range(len(data[i])):
        if(pd.isna(data[i][j])==True):
            row.append(j)
row=set(row) 
data=data.drop(row)
data.info()

FileNotFoundError: [Errno 2] No such file or directory: 'PRML\\penguins.csv'

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
l=[]
for i in data:
    if data.dtypes[i]=='object':
        l.append(i)
        data[i]=le.fit_transform(data[i])    
li=[]
index=0
for i in data:
    if data[i].dtype=='float64':
        li.append(index)
        index+=1
    else:
        index+=1         

In [ ]:
#visualisation
from pandas.plotting import scatter_matrix
scatter_matrix(data[data.columns],figsize=(25,25))


In [ ]:
import seaborn as sns
sns.pairplot(data, hue = "species", palette="Set2")

In [ ]:
sns.relplot(data=data,x="flipper_length_mm", y="bill_depth_mm",hue="species")
sns.relplot(data=data,x="bill_depth_mm", y="bill_length_mm",hue="species")

In [ ]:
#2 cost function
def gini_index(y):
    a,counts = np.unique(y,return_counts=True)

    probabilities = counts/counts.sum()
    gini = sum(probabilities**2)
    return 1-gini
def info_gain(parent,left,right):
    p_left = len(left) / len(parent)
    p_right = len(right) / len(parent)
    return gini_index(parent) - (p_left*gini_index(left) + p_right*gini_index(right))


In [ ]:
#3continuous variables converted to categorical variables first.
def cont_to_cat(y,data1):
    thre=0
    gain=-1000
    j=0
    y1=data1[:,y]
    y1=np.unique(y1)
    while(j<len(y1)):
        left = np.array([row for row in data1 if row[y]<=y1[j]])
        right = np.array([row for row in data1 if row[y]>y1[j]])
        parent=data1[:,0]        
        curr_info_gain =info_gain(parent, left, right)
        if(curr_info_gain>gain):
            gain=curr_info_gain
            thre=y1[j]
        j+=1     
    for row in data1:
        if(row[y]<=thre):
            row[y]=1
        else:
            row[y]=0
    return data1
       
data=np.array(data)
for i in li:
    data=cont_to_cat(i,data)
print(data)    

In [ ]:
#tree formation
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value
def best_split(dataset, num_features):
        bestsplit = {}
        gain = -2000000
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            thresholds = np.unique(feature_values)
            for i in thresholds:
                left = np.array([row for row in dataset if row[feature_index]<=i])
                right = np.array([row for row in dataset if row[feature_index]>i])
                if len(left)>0 and len(right)>0:
                    y, left_y, right_y = dataset[:, -1], left[:, -1], right[:, -1]
                    curr_info_gain = info_gain(y, left_y, right_y)
                    if curr_info_gain>gain:
                        bestsplit["feature_index"] = feature_index
                        bestsplit["threshold"] = i
                        bestsplit["left"] = left
                        bestsplit["right"] = right
                        bestsplit["info_gain"] = curr_info_gain
                        gain = curr_info_gain
        return bestsplit

In [ ]:

def tree(dataset, curr_depth=0,max_depth=10,min_samples_split=3):
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        if num_samples>=min_samples_split and curr_depth<=max_depth:
            bestsplit = best_split(dataset, num_features)
            if(bestsplit!={}):
                if bestsplit["info_gain"]>0:
                    left_subtree = tree(bestsplit["left"], curr_depth+1)
                    right_subtree = tree(bestsplit["right"], curr_depth+1)
                    return Node(bestsplit["feature_index"], bestsplit["threshold"], left_subtree, right_subtree, bestsplit["info_gain"])

        Y = list(Y)
        leaf=max(Y, key=Y.count)
        return Node(value=leaf)
        

In [ ]:
#prediction on testing data and finding its accuracy
def predict(x,root):
    if root.value!=None: 
        return root.value
    feature_val = x[root.feature_index]
    if feature_val<=root.threshold:
        return predict(x, root.left)
    else:
        return predict(x, root.right)

In [ ]:
X = data[:, 1:]
Y = data[:, 0]
Y=Y.reshape(len(Y),1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=47)
dataset = np.concatenate((X_train, Y_train), axis=1)
root =tree(dataset)


In [ ]:
predictions=[predict(i,root) for i in X_test]
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, predictions)

In [ ]:
#class_wise accuracy
c0=0
c0_p=0
c1=0
c1_p=0
c2=0
c2_p=0
for i in range(len(Y_test)):
    if(Y_test[i]==0):
        c0+=1
        if(predictions[i]==0):
            c0_p+=1
    if(Y_test[i]==1):
        c1+=1
        if(predictions[i]==1):
            c1_p+=1        
    if(Y_test[i]==2):
        c2+=1
        if(predictions[i]==2):
            c2_p+=1   
print((c0_p/c0+c1_p/c1+c2_p/c2)/3)

# 2

In [ ]:
#1 preprocessing
data1=pd.read_csv('ENB2012_data.csv')
row=[]
c=data1.loc[:, data1.isnull().any()].columns
for i in c:
    for j in range(len(data1[i])):
        if(pd.isna(data1[i][j])==True):
            row.append(j)
row=set(row) 
data1=data1.drop(row)
data1.info()

In [ ]:
# data splitting
from sklearn.model_selection import train_test_split
def data_splitting(dataset):
    X_train,X_rem=train_test_split(dataset,train_size=0.7,random_state=41)
    X_test,X_valid=train_test_split(X_rem,train_size=0.67,random_state=41)
    return X_train,X_test,X_valid

a,b_test,c=data_splitting(data1)
print(a)
print(b_test)
print(c)

In [ ]:
#2 hyper-parameter tuning 
from sklearn.tree import DecisionTreeRegressor 
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
mse1=[]
max_depth1=[]
md=0
b=10000
for max_depth in range(1,50):
    regressor = DecisionTreeRegressor(max_depth=max_depth) 
    regressor.fit(a.iloc[:,:-1], a.iloc[:,-1])
    y_pred = regressor.predict(c.iloc[:,:-1])
    y_v=np.array(c.iloc[:,-1])
    mse=mean_squared_error(y_v, y_pred)
    mse1.append(mse)
    max_depth1.append(max_depth)
    if(mse<b):
        b=mse
        md=max_depth
plt.figure(figsize=(10,10))
fig2 = plt.plot( max_depth1,mse1, linestyle='solid')
print(b)
plt.ylabel("MSE")
plt.xlabel("max_depth")        

In [ ]:
mse2=[]
max_leaf1=[]
md=0
mln=0
b1=10000
for i in range(2,150):
    regressor = DecisionTreeRegressor(max_leaf_nodes=i) 
    regressor.fit(a.iloc[:,:-1], a.iloc[:,-1])
    y_pred = regressor.predict(c.iloc[:,:-1])
    y_v=np.array(c.iloc[:,-1])
    mse=mean_squared_error(y_v, y_pred)
    mse2.append(mse)
    max_leaf1.append(i)
    if(mse<b1):
        b1=mse
        mln=i
plt.figure(figsize=(10,10))
fig2 = plt.plot( max_leaf1, mse2,linestyle='solid')
print(b1)
plt.ylabel("MSE")
plt.xlabel("max_leaf")

In [ ]:
mse3=[]
min_sample1=[]
mss=0
b2=10000
for i in range(2,50):
    regressor = DecisionTreeRegressor(min_samples_split=i) 
    regressor.fit(a.iloc[:,:-1], a.iloc[:,-1])
    y_pred = regressor.predict(c.iloc[:,:-1])
    y_v=np.array(c.iloc[:,-1])
    mse=mean_squared_error(y_v, y_pred)
    mse3.append(mse)
    min_sample1.append(i)
    if(mse<b2):
        b2=mse
        mss=i
plt.figure(figsize=(10,10))
fig2 = plt.plot( min_sample1,mse3, linestyle='solid')
print(b2)
plt.ylabel("MSE")
plt.xlabel("min_sample")

In [ ]:
mse4=[]
min_leaf1=[]
msl=0
b3=10000
for i in range(2,50):
    regressor = DecisionTreeRegressor(min_samples_leaf=i) 
    regressor.fit(a.iloc[:,:-1], a.iloc[:,-1])
    y_pred = regressor.predict(c.iloc[:,:-1])
    y_v=np.array(c.iloc[:,-1])
    mse=mean_squared_error(y_v, y_pred)
    mse4.append(mse)
    min_leaf1.append(i)
    if(mse<b3):
        b3=mse
        msl=i
plt.figure(figsize=(10,10))
fig2 = plt.plot( min_leaf1,mse4, linestyle='solid')
print(b3)
plt.ylabel("MSE")
plt.xlabel("min_leaf")

In [ ]:
mse5=[]
max_depth2=[]
max_leaf2=[]
min_sample2=[]
min_leaf2=[]
md1=0
mln1=0
mss1=0
msl1=0
b_main=10000
for max_depth in range(3,20):
    for max_leaf in range(30,90):
        for min_sample in range(2,10):
            for min_leaf in range(2,5):
                regressor = DecisionTreeRegressor(max_depth=max_depth,max_leaf_nodes=max_leaf,min_samples_split=min_sample,min_samples_leaf=min_leaf,random_state=41)
                regressor.fit(a.iloc[:,:-1], a.iloc[:,-1])
                y_pred = regressor.predict(c.iloc[:,:-1])
                y_v=np.array(c.iloc[:,-1])
                mse=mean_squared_error(y_v, y_pred)
                mse5.append(mse)
                max_depth2.append(max_depth)
                min_leaf2.append(min_leaf)
                max_leaf2.append(max_leaf)
                min_sample2.append(min_sample)
                if(mse<b_main):
                    b_main=mse
                    md1=max_depth
                    mln1=max_leaf
                    mss1=min_sample
                    msl1=min_leaf
print(b_main)


In [ ]:
#3 5-fold cross-validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
regressor = DecisionTreeRegressor(max_depth=md1,max_leaf_nodes=mln1,min_samples_split=mss1,min_samples_leaf=msl1,random_state=41)
kfold = KFold(n_splits=5, random_state=1, shuffle=True)

scores = cross_val_score(regressor, b_test.iloc[:,:-1], b_test.iloc[:,-1],scoring="neg_mean_squared_error", cv=kfold)
print(scores)


In [ ]:
# image of tree on testing data
from sklearn import tree
regressor.fit(b_test.iloc[:,:-1], b_test.iloc[:,-1])
fig = plt.figure(figsize=(175,175))
a = tree.plot_tree(regressor, filled=True,fontsize=25)